# Data compression fundamentals

### Data? ... which data?

* We will encode audio, image, and video signals.

### Why?

* After the digitalization of any signal we get a sequence $s[]$
  of samples that represent the signal $s$ with more or less fidelity.
  
* $s[]$ is encoded using PCM (Pulse Code Modulation), in which
  every sample is represented with the same number of bits. For
  example, in a CD we have a data-rate of
  
  $$
    (16+16)\frac{\text{bits}}{\text{sample}}\times
    44{.}100\frac{\text{samples}}{\text{second}}=
    1{.}411{.}200\frac{\text{bits}}{\text{second}}.
  $$

### Sources of redundancy in signals

* In general, signals has different types of redundancy:

    + **Statistical redundancy**: It can be removed using
    probabilistic models of the signal producing lossless codecs. The
    codecs are also known as *text codecs*.
    
    + **Spatial/temporal redundancy**: It can be removed using
    spatial/temporal models of the signal and produces also lossless
    codecs.
    
    + **Psychological redundancy**: Some information that
    signal carry can not be perceived by humans. This kind of
    pseudo-redundancy is removed normally by means of quantization,
    producing lossy codecs.

### Symbols, runs, strings, code-words and code-streams

* In the context of statistical coding, each sample of $s[]$ is
  called a *symbol*.
  
* Depending on the type of statistical relationship among
  symbols, we will speak also about *strings* when we process
  more than one symbol and about *runs* when all the symbols are
  the same in a string.
  
* In any case, the output of the encoder is a sequence of
  code-words that all together generates a *code-stream*.

## A. Run-length encoding

* RLE (Run Length Encoding) is a technique that removes the data
  redundancy produced by the repetition of symbols. Example:
  ```
  aaaaa <-> 5a
  ```
  
* There are several versions of RLE codecs, which are different in
  the size of the source alphabet or the maximal/minimal length that
  the runs can take.

### A.1 N-ary run length encoding

RLE for N-ary alphabets (alphabets of size N), where typically, N=256.

#### Encoder

1. While there are symbols to encode:
    1. Let $s$ the next symbol.
    2. Read the next $n$ consecutive symbols equal to $s$.
    3. Write the pair $ns$.

#### Decoder

1. While there are $ns$ pairs to decode:
    1. Write $n$-times the symbol $s$.
    
#### Example

Runs:
```
aaaabbbbbaaaaaabbbbbbbcccccc
```
are encoded as:
```
4a5b6a7b6c
```

#### Lab

In [3]:
# https://rosettacode.org/wiki/Run-length_encoding#Python
    
from itertools import groupby
def encode(input_string):
    return [(len(list(g)), k) for k,g in groupby(input_string)]
 
def decode(lst):
    return ''.join(c * n for n,c in lst)

x = encode('aaaabbbbbaaaaaabbbbbbbcccccc')
print(x)
y = decode(x)
print(y)

[(4, 'a'), (5, 'b'), (6, 'a'), (7, 'b'), (6, 'c')]
aaaabbbbbaaaaaabbbbbbbcccccc


### A.2 Binary run length encoding

* It is not necessary to indicate the next symbol
  (only the length) because if a run ends, the other (possible) symbol
  start with the next run.
  
#### Encoder

1. Let $s\leftarrow$ \texttt{0}.
2. While there are bits to encode:
    1. Read the next $n$ consecutive bits equal to $s$.
    2. Write $n$.
    3. $s\leftarrow (s+1)~\text{modulus}~2$.
    
#### Decoder

1. Let $s\leftarrow$ \texttt{0}.
2. While there are items $n$ to decode:
    1. Write $n$ bits equal to $s$.
    2. $s\leftarrow (s+1)~\text{modulus}~2$.

#### Example

Runs:
```
0000111110000001111111000000
```
are encoded as::
```
4 5 6 7 6
```

#### Lab

In [4]:
# TO-DO

### A.3 [MPN-5 run length encoding](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=held+data+compression+techniques+applications&btnG=)

* Created by Microcom Inc. for the MNP (Microcom Networking
  Protocol) 5.

#### Codec

* The behavior of the codec can be easily defined with the
  following examples:
  
```
Input     Output
--------- ---------
ab        ab
aab       aab
aaab      aaa0b
aaaab     aaa1b
aaaaab    aaa2b
:         :
a^nb      aaa(n-3)b
```

#### Example

Runs:
```
aaaabbbbbaaaaaabbbbbbbcccccc
```
are encoded as:
```
aaa1bbb2aaa3bbb4
```

#### Lab

In [ ]:
# TO-DO

## A.4 [Burrows-Wheeler transform](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Burrows+M%2C+Wheeler+DJ%3A+A+Block+Sorting+Lossless+Data+Compression+Algorithm.&btnG=)

* BWT (Burrows-Wheeler Transform) is an algorithm that inputs
  a string and outputs:
  1. A different string with the same symbols (with longer runs),
    but with a different order.
  2. An index.
  
  
* There is an inverse transform that, using the output of the
  forward transform, recover the original string.
  
* The transformed string tends to have longer runs.

* The length of the runs in proportional to the correlation
  between the symbols and the length of the input.
  
### Forward transform

Let $B$ the block-size in symbols:

1. Read $B$ symbols.
2. Build a square matrix of size $B\times B$ where the first row is
  the original sequence, the second one is the same sequence but
  cyclically shifted one symbol to the left, and so on ...
3. Sort lexicographically the matrix by rows.
4. Search in the last column the row in which the first symbol of
  the original sequence it is found. This is the index $i$.
5. Output $i$ and the last column $O[]$.

#### Encoding example
<img src="text_coding/BWT_example.svg" style="width: 800px;"/>

### Inverse transform

1. Sort $O[]$ over $S[]$.
2. Compute $T[]$ where if $S[j]=O[l]$ (being $l$ the first symbol
  of $O[]$ that matches this condition), then $T[j]=l$. Notice that
  all of symbols of $T[]$ have to be different.
3. Let $k\leftarrow i$.
4. Execute $B$ times:
    1. Output $O[k]$.
    2. $k\leftarrow T[k]$.
    
#### Decoding example
<img src="text_coding/BWT_example_decod.svg" style="width: 400px;"/>

### Lab

In [7]:
# https://gist.github.com/dmckean/9723bc06254809e9068f

def bwt_encode(s):
    n = len(s)
    m = sorted([s[i:n]+s[0:i] for i in range(n)])
    I = m.index(s)
    L = ''.join([q[-1] for q in m])
    return (I, L)

from operator import itemgetter

def bwt_decode(I, L):
    n = len(L)
    X = sorted([(i, x) for i, x in enumerate(L)], key=itemgetter(1))

    T = [None for i in range(n)]
    for i, y in enumerate(X):
        j, _ = y
        T[j] = i

    Tx = [I]
    for i in range(1, n):
        Tx.append(T[Tx[i-1]])

    S = [L[i] for i in Tx]
    S.reverse()
    return ''.join(S)

index, encoded = bwt_encode('ababcbababaaaaaaa')
print (index, encoded)
decoded = bwt_decode(index, encoded)
print (decoded)

9 baaaaaabbabaacaab
ababcbababaaaaaaa


## B. String encoding

### How it works?

* We replace strings by code-words of less length.
* Strings are searched in a dictionary and the sequence of positions of the strings in the dictionary form the code-strem.

### B.1 LZ77 [[J. Ziv and A. Lempel, 1977]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Ziv+Lempel+universal+sequential+data+compression+1977&btnG=)

* In 1977, Jacov Ziv and Abraham Lempel propose the LZ77 algorithm.
* In the eighties, a branch of LZ77 known as LZSS and is
  implemented by Haruyasu Yoshizaki in the program LHARC, discovering
  the possibilities of the LZ77 encoding.
* After that, a large number of text compressors have been based
  on the LZ77 idea (or a variation of it). Some of the most famous
  are: `ARJ`, `RAR`, `gzip` and `7z`.
* LZ77 processes a sequence of symbols using the structure:

<img src="text_coding/LZ77.svg" style="width: 600px;"/>

* The dictionary and the look-ahead buffer have a fixed size and
  can be considered as a sliding window, where the input of a new
  symbol generates the output of the oldest one, which becomes the
  newest symbol of the dictionary.
  
#### Encoder

1. Let $I$ the length of the dictionary and $J$ the length of the
  buffer.
2. Input the first $J$ symbols in the buffer.
3. While the input is not exhausted:
    1. Let $i$ the position in the dictionary of the first $j$
    symbols of the buffer and $k$ the symbol that makes that $j$ can
    not be larger.
    2. Output $ijk$.
    3. Input the next $j+1$ in the buffer.
    
#### Decoder

1. While the code-words $ijk$ are not exhausted:
    1. Output the $j$ symbols extracted from the position $i$ in the
    dictionary.
    2. Output $k$.
    3. Introduce all the decoded symbols into the buffer.

#### Example

<img src="text_coding/LZ77_encoding_example.svg" style="width: 600px;"/>

<img src="text_coding/LZ77_decoding_example.svg" style="width: 600px;"/>

* Parameters $I$ and $J$ control the performance
  of the algorithm. They should be large enough to guarantee the
  matching of long strings, but should keep small in order to reduce
  the number of bits of the code-words $ijk$. Typical sizes are:
  $\log_2(I)=12.0$ and $\log_2(J)=4.0$.

#### Lab

In [5]:
# To-do.

### B.2 LZ78 [[J. Ziv and A. Lempel, 1978]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Ziv+Lempel+1978&btnG=)

* In 1978, Ziv and Lempel published the LZ78 algorithm.

* LZ89 represents the dictionary in a recursive way with the idea
  of improving the search of the strings in the dictionary. Now, each
  entry in the dictionary is a pair $wk$, where $w$ is a pointer to
  the dictionary and $k$ is a symbol. In fact, each entry $wk$
  represents the string that results from the concatenation of string
  $w$ and $k$, where $w$ can be recursively computed as we have found
  $wk$.
  
* We will denote \textit{string}$(w)$ to the string that $w$
  represents.
  
* The empty string is obtained by \textit{string}$(0)$.

#### Encoder

1. $w\leftarrow 0$.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $wk$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow 0$.
        
#### Decoder

1. While the input is not exhausted:
    1. Input $wk$.
    2. Output $\text{string}(w)$.
    3. Output $k$.
    4. Insert $wk$ in the dictionary.
    
#### Example

<img src="text_coding/LZ78_encoding_example.svg" style="width: 600px;"/>

<img src="text_coding/LZ78_decoding_example.svg" style="width: 600px;"/>

#### Lab

In [ ]:
# TO-DO

### B.3 LZW [[T.A. Welch, 1984]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Terry+Welch+1984&btnG=)

* In 1984 Terry A. Welch proposes the LZW algorithm,
  which is an improved version of the LZ89 algorithm that does not
  writes raw symbols ($k$) to the code-stream.

* LZW was selected as encoding engine for the GIF (Graphics
  Interchange Format), and for the compressor `compress`.
  
* The dictionary is initially filled with the $2^k$ possible
  symbols (*roots*), that are stored in entries $0\cdots255$.
  
#### Encoder

1. $w\leftarrow$ next input symbol.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $\leftarrow w$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow k$.

#### Decoder

1. $code\leftarrow$ first input code-word.
2. Output $code$.
3. $old\_code\leftarrow code$.
4. While the input is not exhausted:
    1. $code\leftarrow$ next input code-word.
    2. $w\leftarrow old\_code$.
    3. If $code$ is found in the dictionary, then:
        1. Output string$(code)$.
    4. Else:
        1. Output string$(w)$.
        2. Output $k$.
    5. $k\leftarrow$ first symbol of the last output.
    6. Insert $wk$ in the dictionary.
    7. $old\_code\leftarrow code$.
    
#### Example

<img src="text_coding/LZW_encoding_example.svg" style="width: 600px;"/>

<img src="text_coding/LZW_decoding_example.svg" style="width: 600px;"/>


#### Lab

In [8]:
# https://rosettacode.org/wiki/LZW_compression#Python

# TO-DO

## C. Symbol encoding

### How it works?

* We can compress if each symbol is translated by code-words and,
  in average, the lengths of the code-words are smaller than the
  length of the symbols.
  
* The encoder and the decoder have a probabilistic model $M$ which
  says to the variable-length encoder ($C$)/decoder($C^{-1}$) the
  probability $p(s)$ of each symbol $s$.
  
<img src="text_coding/compresion_entropica.svg" style="width: 600px;"/>

* The most probable symbols are represented by the shorter
  code-words and viceversa.
  
### Bits

* Data is the representation of the information.

* Lossless data compression uses a shorter representation of the
  information.
  
* By definition, a bit of data stores a bit of information if and
  only if it represents the occurrence of an equiprobable event (an
  event that can be true or false with the same probability).
  
* By definition, a symbol $s$ with probability $p(s)$ stores

\begin{equation}
  I(s)=-\log_2 p(s) \tag{Eq:symbol_information}
  \label{eq:symbol_information}
\end{equation}
  
  bits of information.

* The length of the code-word depends on the probability as:

<img src="text_coding/prob_vs_long.svg" style="width: 600px;"/>

### Entropy

* The entropy $H(S)$ measures the amount of information per
  symbol that a source of information $S$ produces, in average, i.e.
  
\begin{equation}
  H(S) = \frac{1}{N}\sum_{s=1}^{N} p(s)\times I(s)
\end{equation}

  bits-of-information/symbol, where $N$ is the size of the source
  alphabet (number of different symbols).

### C.1 Universal coding

* An ideal entropy encoder should represent each symbol $s$ with a
  number of bits that $I(s)$ says (see \ref{eq:symbol_information}).
  
* This system will be 100\% efficient is the guesses are
  equiprobable!
  
#### Encoding of a symbol

1. While the decoder does not know the symbol:
    1. Assert something about the symbol that allows to the decoder
    to minimize the uncertainty of finding that symbol. This guess
    should have the same probability of to be true or false.
    2. Output a bit of code that says if the last guess is true or
    false.
    
#### Decoding of a symbol

1. While the symbol is not known without uncertainty:
    1. Make the same guess that the encoder.
    2. Input a bit of code that represents the result of the last
    guess.

#### Example

* Let's suppose that we use the Spanish alphabet. Humans know that
  symbols does not form words in any order (this fact can help us to
  formulate the following VLC (Variable Length Codec)):
  
* In Spanish there are 28 letters. Therefore, to encode, for example,
  the word "preciosa", the first symbol ``p'' can be represented by
  it index inside the Spahish alphabet with a code-word of 5 bits. In
  this try, the encoding is not a very efficient, but this one is the
  first letter ... For the second one ``r'' we can see (using a
  Spanish dictionary) that after a ``p'', the following symbols are
  possible: (1) ``a'', (2) ``e'', (3) ``i'', (4) ``l'', (5) ``n'', (5)
  ``o'', (7) ``r'', (8) ``s'' and (9) ``u''. Therefore, we don't need
  5 bits now, 4 are enough.
  
<img src="text_coding/universal_coding_example.svg" style="width: 300px;"/>

* The compression ratio has been 40/25:1 (``preciosa'' has 8
  letters).

### C.2 Shannon-Fano coding

* At the end of the 40's, Claude E. Shannon (Bell Labs) and
  R.M. Fano (MIT) developed an VLC codec.
  
#### Encoder

1. Sort the symbols using their probabilities.
2. Split the set of symbols into two subsets in a way in which the
   each subset have the same total probability.
3. Assign a different bit to each set.
4. Repeat the previous procedure to each subset until the size of
   each subset is equal to 1.
   
#### Decoder

To-do.

#### Example

* Let's use the next probabilistic model:
<img src="text_coding/shannon-fano_example.svg" style="width: 120px;"/>
Using it, this is the coding:
<img src="text_coding/shannon-fano_example-coding.svg" style="width: 850px;"/>





### C.3 Huffman coding [[Huffman, 1952](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=huffman+method+codes+1952&btnG=)
 
* Optimal performance (better than Shannon-Fano) when a integer
  number of bits is assigned to each symbol.
* The VLC codec builds a binary tree where the symbols are stored
  in the leafs and the distance of each symbol to the root of the tree
  is $\lceil\log_2(p(s))\rceil$.
* After label each binary branch in the tree, the Huffman
  code-word for the symbol $s$ is the sequence of bits (labels) that
  we must use to travel from the root to the $s$-leaf.
 
#### Building Huffman trees

1. Create a list of nodes. Each node stores a symbol and its
   probability.
2. While the number of nodes in the list > 1:
    1. Extract from the list the 2 nodes with the lowest probability.
    2. Insert in the list a new node (that is the root of a binary
       tree) whose probability is the sum of the probability of its
       leafs.
       
#### Example

<img src="text_coding/huffman_ejemplo.svg" style="width: 800px;"/>

#### Limits

* Any Huffman code satisfies that
  \begin{equation}
    l\big(c(s)\big) = \lceil I(s)\rceil, \tag{Eq:Huffman}
    \label{eq:Huffman}
  \end{equation}
  where $l\big(c(s)\big)$ is the length of the code-word assigned to
  the symbol $s$.
  
* This implies that, with every encoded symbol, up to 1 bit of
  redundant data can be introduced.
  
* This is a problem that grows when the size of the alphabet is
  small. In the extreme case, for binary source alphabets, the Huffman
  coding does not change the length of the original representation.

### C.4 Arithmetic coding

* Arithmetic coding relaxes the Equation \ref{eq:Huffman},
  verifying that, for every encoded symbol, 
  \begin{equation}
    l\big(c(s)\big) = I(s), \tag{Eq:arithmetic}
    \label{eq:arithmetic}
  \end{equation}
  i.e. the number of bits of data (code-word) assigned by the encoder
  is equal to the number of bits of information that the symbol
  represent.

<img src="text_coding/comparacion.svg" style="width: 800px;"/>

* It can be said that, the arithmetic coding is optimal because
  the average length of an arithmetic code is equal to the entropy of
  the information source, measured in bits/symbol.

#### Ideal encoder

1. Let $[L,H)\leftarrow [0.0,1.0)$ an interval of real numbers.
2. While the input is not exhausted:
    1. Split $[L,H)$ into so many sub-intervals as different symbols
       are in the alphabet. The size of each sub-interval is proportional
       to the probability of the corresponding symbol.
    2. Select the sub-interval $[L',H')$ associated with the encoded
       symbol.
    3. $[L,H)\leftarrow [L',H')$.
3. Output a real number $x\in[L,H)$ (the arithmetic
   code-stream). The number of decimals of $x$ should be large enough
   to distinguish the final sub-interval $[L,H)$ from the rest of
   possibilities.
   
#### Example

* Imagine a binary sequence, where $p(\text{A})=3/4$ and
  $p(\text{B})=1/4$. Compute the arithmetic code of the sequences A, B,
  AA, AB, BA y BB.
  
<img src="text_coding/aritmetica_ejemplo.svg" style="width: 500px;"/>

#### Incremental transmission

* It is not necessary to wait for the end of the encoding to
  generate the arithmetic code. When we work with binary
  representations of the real numbers $L$ and $H$, their most
  significant bits are identical as the interval is reduced. These
  bits are going to contribute to the arithmetic code, therefore, they
  can be output as soon as they are known.
  
  For example, when the symbol B is encoded, a code-bit 1 can be
    output because any sequence of symbols that start with B have a
    code-word that begins by 1.
    
* When the most significant bits of $L$ and $H$ are output, the
  bits of each register are shifted to the left, and new bits need to
  be inserted. The results is an automatic zoom of the selected
  sub-interval.

  Following with the previous example, the register shifting generates
    an ampliation of the $[0.50,1.00)$ interval to the $[0.00,1.00)$.
    
#### The ideal decoder

1. Let $[L,H)\leftarrow [0.0,1.0)$ the initial interval.
2. While the input is not exhausted:
    1. Split $[L,H)$ into so many sub-intervals as different symbols
       are in the alphabet. The size of each sub-interval is proportional
       to the probability of the corresponding symbol.
    2. Input so many bits of $x$ as they are needed to:
        1. Select the sub-interval $[L',H')$ that contains $x$.
        2. Output the symbol that $[L',H')$ represents.
        3. $[L,H)\leftarrow[L',H')$.

## C.5 Probabilistic models

* In order to use any of the previous VLCs, a probabilistic model is needed.

### Static models

* The simplest models because the probabilities of the symbols
  remain constant.
* The variable-length codec can be precomputed.
* If the last premise is true, the entropy codec is efficient an
  fast. For this reason, static models are very common in codecs such
  as JPEG, MPEG (audio and video), etc.
  
### Adaptive models

* The probabilities of the symbols are computed in run-time.
* In general, the compression ratios that an adaptive model 
  get are better than the static model's ones because the
  probabilities of the symbols are localy computed.
  
#### Encoding

1. Asign the same probability to all the symbols.
2. While the input if not exhausted:
    1. Encode the next symbol.
    2. Update (increase) its probability.

#### Decoding

1. Identical to the step 1 of the encoder.
2. While the input is not exhausted:
    1. Decode the next symbol.
    2. Identical to the step 2.b of the encoder.
    
### Initially empty models

* The smaller the number of symbols used by the model, the higher
  the probabilities, and therefore, the better the compression ratios.
* An initially empty model only stores the ESC(cape) symbol, a
  symbol that it is used by the encoder only when a new symbol is
  found.

#### Encoder

1. Set the probability of the ESC to $1.0$ (and the probability of
   the rest of the symbols to $0.0$).
2. While the input is not exhausted:
    1. $s\leftarrow$ next symbol.
    2. If $s$ has been found before, then:
        1. Encode $s$ and output $c(s)$.
    3. Else:
        1. Output $c(\mathrm{ESC})$.
        2. Output a raw symbol $s$.
    4. Update $p(s)$.
    
#### Decoder

1. Identical to the step 1 of the encoder.
2. While the input is not exhausted:
    1. $c(s)\leftarrow $ next code-word.
    2. Decode $s$.
    3. If $s=$ ESC, then:
        1. Input a raw symbol $s$.
    4. Update $p(s)$.
    5. Output $s$.
    
### Models with memory

* In most cases, the probability of a symbol depends on its
  neighborhood (context).
* The higher the memory of the model (the context), the higher the
  accuracy of the predictions (probabilities), and therefore, the
  lower the length of the code-words \cite{Cleary.PPM}.
* Let ${\cal C}[i]$ the last $i$ encoded symbols and
  $p(s|{\cal C}[i])$ the probability that the symbol $s$ follows
  the context ${\cal C}[i]$.
* Let $k$ the maximal order of the prediction (i.e. the largest
  number of symbols of ${\cal C}[]$ that are going to be used as the
  actual context). Notice that ${\cal C}[0]=\varnothing$ and the model
  has no memory.
* We suppose that arithmetic coding is used and therefore, when we
  input or output $c(s)$, we are transmitting $I(s)$ bits of code.
* Let $r$ the size of the source alphabet.

#### Encoder

1. Create an empty model for every context $0\le i \le k$.
2. Create an non-empty model for $k=-1$.
3. While the input is not exhausted:
    1. $s\leftarrow$ Input$_{\log_2(r)}$.
    2. $i\leftarrow k$ (except for the first symbol, where
       $i\leftarrow 0$).
    3. While $p(s|{\cal C}[i])=0$ (it is the first time that $s$ follows
       ${\cal C}[i]$):
        1. Output $\leftarrow c(\text{ESC}|{\cal C}[i])$.
        2. Update $p(\text{ESC}|{\cal C}[i])$.
        3. Update $p(s|{\cal C}[i])$ (insert $s$ into the ${\cal C}[i]$ context).
        4. $i\leftarrow i-1$.
    4. Output $\leftarrow c(s|{\cal C}[i])$. The symbols that were in
       contexts with order $>i$ must be excluded of the actual (${\cal C}[i]$) context because $s$ is not none of them.
    5. If $i\ge 0$, update $p(s|{\cal C}[i])$.
    
#### Example

* Let $r=256$ the size of the source alphabet.

* The probabilistic model $M[{\cal C}[-1]]$ (for the special context
  ${\cal C}[-1]$) is non adaptative, non empty and has an special symbol EOF
  (End Of File) that is going to be used when the compression has
  finished:
  $$M[{\cal C}[-1]]=\{0,1~1,1~\cdots~\mathtt{a},1~\mathtt{b},1~\cdots~255,1~\text{EOF},1\}.$$
  In a pair $a,b$, $a$ is the symbol and $b$ is its probability (counts).

* $M[{\cal C}[0]]$ is adaptative and empty:
  $$M[{\cal C}[0]]=\{\text{ESC},1\}.$$

* In this example (for the sake of the simplicity), the maximal
  order of the prediction $k=1$ (we only remember the previous
  symbol). Therefore, there are $r=256$ probabilistic models:
  $$M[{\cal C}[1]]=\{\text{ESC},1\}, 0\le {\cal C}[1]\le r.$$
  
* Encoding of the first symbol (\texttt{a}) (see Figure~\ref{fig:PPM}):

1. [3.A] $s\leftarrow$ \texttt{a}.
2. [3.B] $i\leftarrow 0$ (we don't know the previous symbol).
3. [3.C] $p(\mathtt{a}|{\cal C}[0])=0$ (the context only has the ESC).
4. [3.C.a] Output $\leftarrow c(\text{ESC}|{\cal C}[0])$ (althought
    $l(c(\text{ESC}|{\cal C}[0]))=0$).
5. [3.C.b] Update $p(\text{ESC}|{\cal C}[0])$ (now, the count of ESC is
    2).
6. [3.C.c] Insert \texttt{a} into
    $M[{\cal C}[0]]=\{\mathsf{ESC},2~\mathtt{a},1\}$.
7. [3.C.d] $i\leftarrow -1$.
8. [3.c] $p(\mathtt{a}|{\cal C}[-1])\neq 0$.
9. [3.d] Output $\leftarrow c(\texttt{a}|{\cal C}[-1])$ where
    $p(\texttt{a}|{\cal C}[-1]) = 1/(256+1)$.
    
* Encoding of the second symbol (\texttt{b}):

1. [3.a] $s\leftarrow$ \texttt{b}.
2. [3.b] $i\leftarrow 1$.
3. [3.c] $p(\mathtt{b}|{\cal C}[1])=0$ because ${\cal C}[1]=\texttt{a}$ and
   $M[\texttt{a}]=\{\text{ESC},1\}$.
4. [3.c.i] Output $\leftarrow c(\text{ESC}|\texttt{a})$ (althought
   $l(c(\text{ESC}|\texttt{a}))=0$).
5. [3.c.ii] Update $p(\text{ESC}|\texttt{a})$ (now, the count of ESC is 2).
6. [3.c.iii] Insert \texttt{b} into $M[\texttt{a}]=\{\text{ESC},2~ \texttt{b},1\}$.
7. [3.c.iv] $i\leftarrow 0$.
8. [3.c] $p(\mathtt{b}|{\cal C}[0])=0$ because
   $M[{\cal C}[0]]=\{\mathsf{ESC},2~\texttt{a},1\}$.
9. [3.c.i] Output $\leftarrow c(\text{ESC}|{\cal C}[0])$ where
   $p(\text{ESC}|{\cal C}[0]) = 2/3$.
10. [3.c.ii] Update $p(\text{ESC}|{\cal C}[0])$ (now, the count of ESC is
    3).
11. [3.c.iii] Insert \texttt{b} into $M[{\cal C}[0]] = \{\text{ESC},3~
    \texttt{a},1~ \texttt{b},1\}$.
12. [3.c.iv] $i\leftarrow -1$.
13. [3.c] $p(\mathtt{b}|{\cal C}[-1])\neq 0$.
14. [3.d] Output $\leftarrow c(\texttt{b}|{\cal C}[-1])$ where
    $p(\mathtt{b}|{\cal C}[-1]) = 1/r$. The symbol \texttt{a} has been
    excluded in the calculus of the probability of \texttt{b} because
    $\texttt{a}\in M[{\cal C}[0]] = \{\text{ESC},3~ \texttt{a},1~
    \texttt{b},1\}$.

<img src="text_coding/PPM_example.svg" style="width: 800px;"/>

#### Decoder

1. Equal to the step 1 of the encoder.
2. While the input is not exhausted:
    1. $i\leftarrow k$ (except for the first symbol, where $i\leftarrow 0$).
    2. $s\leftarrow$ next decoded symbol.
    3. While $s=\text{ESC}$:
        1. Update $p(\text{ESC}|{\cal C}[i])$.
        2. $i\leftarrow i-1$.
        3. $s\leftarrow$ next decoded symbol.
    4. Update $p(s|{\cal C}[i])$.
    5. While $i<k$:
        1. $i\leftarrow i+1$.
        2. Update $p(s|{\cal C}[i])$ (insert $s$ into the ${\cal C}[i]$ context).


## C.6 MTF (Move To Front) transform

* Inputs a sequence of symbols and outputs a sequence of symbols.

* The size (in bits of data) for each sequence is the same.

* The entropy of the output is lower that the input's one.

* Performs a change in the representation of the symbols where
  those symbols that have a high probability of occurrency are
  ``moved'' in the source alphabet towards decreasing positions.

* The probability density function follows an exponential
  distribution with a slope $\lambda$ where
\begin{equation}
  f(x.\lambda) = \left\{ \begin{array}{ll}
      \lambda e^{-\lambda x} & \mbox{if $x \geq 0$};\\
      0 & \mbox{if $x < 0$}.\end{array} \right.
\end{equation}

<img src="text_coding/exponential.svg" style="width: 600px;"/>

### Forward transform

1. Create a list $L$ with the symbols of the source alphabet
  where $$L[s]\leftarrow s; 0\le s\le r.$$
2. While the input is not exhausted:
    1. $s\leftarrow$ next input symbol.
    2. $c\leftarrow$ position of $s$ in $L$ ($L[c]=s$).
    3. Output $\leftarrow c$.
    4. Move $s$ to the front of $L$.
    
### Inverse transform

1. The step 1 of the forward transform.
2. While the input is not exausted:
    1. $c\leftarrow$ next input code.
    2. $s\leftarrow L[c]$.
    3.  Output $s$.
    4. The step 2.C of the forward transform.
    
#### Example

<img src="text_coding/MTF_example.svg" style="width: 250px;"/>



## C.7 Context-based Text Predictive transform

* The MTF uses a model where a symbol that has happened only one
  time can get a index-code that is lower than the index-code of a
  symbol that has been found thousands of times :-(

* We can solve this problem if the positions of the symbols are
  determined by their probability. In other words, the list $L$ will
  be sorted by the ocurrence of the symbols.
  
### 0-order encoder

1. The step 1 of the MTF transform, although now every node of the
   list stores also a count of the symbol.
2. While the input is not exhausted:
    1. $s\leftarrow$ next input symbol.
    2. $c\leftarrow$ position of $s$ in $L$ (the prediction error).
    3. Output $\leftarrow c$.
    4. Update the count of $L[c]$ (the count of $s$) and keep sorted $L$.

#### Example

<img src="text_coding/TPT_example.svg" style="width: 450px;"/>

### 0-order decoder

1. The step 1 of the encoder.
2. While the input is not exhausted:
    1. $c\leftarrow$ next input code.
    2. $s\leftarrow L[c]$.
    3. Output $s$.
    4. Step 2.D of the encoder.
    
### $N$-order encoder

1. Let ${\cal C}[i]$ the context of $s$ and $L_{{\cal C}[i]}$ the
   list for that context. If $i>0$ then the lists are empty, else, the
   list is full and the count of every node is $0$.
2. Let $N$ the order of the prediction.
3. Let $H=\varnothing$ a list of tested symbols. All symbols in $H$
   must be different.
4. While the input is not exhausted:
    1. $s\leftarrow$ the next input symbol.
    2. $i\leftarrow k$ (except for the first symbol, where $i\leftarrow 0$).
    3. While $s\notin L_{{\cal C}[i]}$:
        1. $H\leftarrow \text{reduce}(H\cup L_{{\cal C}[i]})$. (reduce$()$ deletes the repeated nodes).
        2. Update the count of $s$ in $L_{{\cal C}[i]}$ and keep sorted it.
        3. $i\leftarrow i-1$.
    4. Let $c$ the position of $s$ en $L_{{\cal C}[i]}$.
    5. $c\leftarrow c+$ symbols of $H-L_{{\cal C}[i]}$. In this
       way, the decoder will know the length of the context where $s$
       happens and does not count the same symbol twice.
    6. Output $\leftarrow c$.
    7. Update the count of $s$ in $L_{{\cal C}[i]}$ and keep sorted it.
    8. $H\leftarrow\varnothing$.
    
#### Example ($k=1$)

<img src="text_coding/TPT_example.svg" style="width: 450px;"/>

### $N$-order decoder

1. Steps 1, 2 and 3 of the encoder.
2. While the input is not exhausted:
    1. $c\leftarrow$ the next input code.
    2. $i\leftarrow k$ (except for the first symbol, where $i\leftarrow 0$).
    3. While $L_{{\cal C}[i]}[c]=\varnothing$:
        1. $H\leftarrow \text{reduce}(H\cup L_{{\cal C}[i]})$.
        2. $i\leftarrow i-1$.
    4. $s\leftarrow \text{reduce}(H\cup L_{{\cal C}[i]})[c]$.
    5. Update the count of $L_{{\cal C}[i]}[c]$.
    6. While $i<k$:
        1. $i\leftarrow i+1$.
        2. Insert the symbol $s$ in $L_{{\cal C}[i]}$.

## C.8 Unary coding

* It is a particular case of the Huffman code where the number of
  bits of each code-word (minus one) is equal to the index of the
  symbol in the source alphabet. Example:
  
<img src="text_coding/Unary_example.svg" style="width: 100px;"/>

* The unary coding is only optimal when (see Equation
  \ref{eq:symbol_information})
  \begin{equation}
    p(s) = 2^{-(s+1)} \tag{Eq:Unary}
    \label{eq:unary}
  \end{equation}
  where $s=0,1,\cdots$.
  
<img src="text_coding/unary.svg" style="width: 800px;"/>



## C.9 Golomb coding [[Golomb, 1966]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=golomb+1966+run&btnG=)

* When the probabilities of the symbols follow an exponential
  distribution, the Golomg encoder has the same efficiency than the
  Huffman coding, but it is faster. In this case, the probabilities of
  the symbols shoud be
  
  \begin{equation}
    p(s) =
    2^{\displaystyle-\Big(\displaystyle m\big\lfloor\displaystyle\frac{s+m}{m}\big\rfloor\Big)}
    \tag{Eq:Golomb}
    \label{eqw:Golomb}
  \end{equation}
  where $s=0,1,\cdots$ is the symbol and $m=0,1,\cdots$ is the
  ``Golomb slope'' of the distribution.
  
* For $m=2^k$, it is possible to find a very efficient
  implementation and the encoder is also named Rice
  encoder~\cite{Rice79}. In this case
  
  \begin{equation}
    p(s) =
    2^{\displaystyle-\Big(2^k \displaystyle\big\lfloor\displaystyle\frac{s+2^k}{2^k}\big\rfloor\Big)}
    \tag{Eq:Rice}
    \label{eq:Rice}
  \end{equation}

<img src="text_coding/Golomb_coding.svg" style="width: 600px;"/>

* Notice that for $m=1$, we take the unary encoding.

<img src="text_coding/Golomb.svg" style="width: 600px;"/>

### Encoder

1. $k\leftarrow \lceil\log_2(m)\rceil$.
2. $r\leftarrow s~\mathrm{mod}~m$.
3. $t\leftarrow 2^k-m$.
4. Output $(s~\mathrm{div}~m)$ using an unary code.
5. If $r<t$:
    1. Output the integer encoded in the $k-1$ least significant bits of $r$ using a binary code.
6. Else:
    1. $r\leftarrow r+t$.
    2. Output the integer encoded in the $k$ least significant bits of $r$ using a binary code.

#### Example ($m=7$ and $s=8$)

1. [1] $k\leftarrow \lceil\log_2(8)\rceil=3$.
2. [2] $r\leftarrow 8 \text{mod} 7 = 1$.
3. [3] $t\leftarrow 2^3-7 = 8-7 = 1$.
4. [4] Output $\leftarrow 8 \text{div} 7 = \lfloor 8/7\rfloor=1$ as an unary code (2 bits). Therefore, output $\leftarrow 10$.
5. [5] $r=1\le t=1$.
6. [6.A] $r\leftarrow 1+1=2$.
7. [6.B] Output $r=2$ using a binary code of $k=3$ bits. Therefore, $c(8)=10010$.

### Decoder

1. $k\leftarrow\lceil\log_2(m)\rceil$.
2. $t\leftarrow 2^k-m$.
3. Let $s\leftarrow$ the number of consecutive ones in the input (we stop when we read a $0$).
4. Let $x\leftarrow$ the next $k-1$ bits in the input.
5. If $x<t$:
    1. $s\leftarrow s\times m+x$.
6. Else:
    1. $x\leftarrow x\times 2~+$ next input bit.
    2. $s\leftarrow s\times m+x-t$.
    
#### Example (decode $10010$ where $m=7$)

1. [1] $k\leftarrow 3$.
2. [2] $t\leftarrow 2^k-m = 2^3-7=1$).
3. [3] $s\leftarrow 1$ because we found only one $1$ in the input.
4. [4] $x\leftarrow \text{input}_{k-1} = \text{input}_2 = 01$.
5. [5] $x=1\nless t=1$.
6. [6.A] $x\leftarrow x\times x\times 2+\text{next input bit} = x\times 1\times 2+0 = 2$.
7. [6.B] $s\leftarrow s\times m+x-t = 1\times 7+2-1=8$.


## C.10 Rice coding

### Encoder

1. $m\leftarrow 2^k$.
2. Output $\leftarrow\lfloor s/m\rfloor$ using an unary code ($\lfloor s/m\rfloor+1$ bits).
3. Output $\leftarrow$ the $k$ least significant bits of $s$ using a binary code.
    
#### Example ($k=1$ and $s=7$)
1. [1] $m\leftarrow 2^k=2^1=2$.
2. [2] Output $\leftarrow \lfloor s/m\rfloor=\lfloor 7/2\rfloor=3$ using an unary code of 4 bits. Therefore, output $\leftarrow 1110$.
3. Output $\leftarrow$ the $k=1$ least significant bits of $s=7$
  using a unary code ($k+1$ bits). So, output $\leftarrow 1$. Total
  output $c(7)=11101$.

### Decoder

1. Let $s$ the number of consecutive ones in the input (we stop when we read a 0).
2. Let $x$ the next $k$ input bits.
3. $s\leftarrow s\times 2^k+x$.

#### Example (decode $11101$ where $k=1$)
1. [1] $s\leftarrow 3$ because we found $3$ consecutive ones in the input.
2. [2] $x\leftarrow$ next input $k=1$ input bits. Therefore $x\leftarrow 1$.
3. [3] $s\leftarrow s\times 2^k+x = 3\times 2^1+1 = 6+1 = 7$.

In [ ]:
#-------------



<img src="text_coding/lena-gray.png" style="width: 400px;"/>
<img src="text_coding/peppers-gray.png" style="width: 400px;"/>
<img src="text_coding/boats.png" style="width: 400px;"/>
<img src="text_coding/zelda.png" style="width: 400px;"/>

Using `rle`, compute the compression ratio of each image as

$$
\gamma = \frac{X}{Y}
$$

where $X$ is the size of the input (the sequence of symbols) and $Y$
the size of the output (the code-stream), and populate:
```
Codec | lena boats pepers zelda Average
------+--------------------------------
  rle | ....  ....   ....  ....    ....
```